## Day 47 Lecture 1 Assignment

In this assignment, we will apply k-means clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

<IPython.core.display.Javascript object>

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [4]:
# answer goes here
nba_df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')
nba_df.head()



,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,...,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,...,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,...,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9


<IPython.core.display.Javascript object>

In [15]:
nba_df

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,...,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,...,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,...,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,2016,Joe Young,PG,23,IND,41,0,9.4,1.5,4.1,...,0.800,0.1,1.1,1.2,1.6,0.4,0.0,0.8,0.7,3.8
2137,2016,Nick Young,SG,30,LAL,54,2,19.1,2.3,6.9,...,0.829,0.3,1.5,1.8,0.6,0.4,0.1,0.6,0.9,7.3
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,...,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1
2139,2016,Cody Zeller,C,23,CHO,73,60,24.3,3.2,6.0,...,0.754,1.9,4.3,6.2,1.0,0.8,0.9,0.9,2.8,8.7


<IPython.core.display.Javascript object>

The goal is to cluster these player-seasons to identify potential player "archetypes".

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [7]:
# answer goes here

# columns to filter on: GS: games started, MP: minutes played
nba_df.GS

0        2
1        0
2        1
3       80
4       28
        ..
2136     0
2137     2
2138    73
2139    60
2140     3
Name: GS, Length: 2141, dtype: int64

<IPython.core.display.Javascript object>

In [8]:
nba_df.MP

0       19.0
1       12.3
2       12.6
3       33.4
4       23.3
        ... 
2136     9.4
2137    19.1
2138    33.0
2139    24.3
2140    11.8
Name: MP, Length: 2141, dtype: float64

<IPython.core.display.Javascript object>

In [14]:
nba_df2 = nba_df[nba_df.GS >= 20]
nba_df2 = nba_df2[nba_df2.MP >= 10]
nba_df2

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
7,2019,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,...,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
10,2019,Jarrett Allen,C,20,BRK,80,80,26.2,4.2,7.1,...,0.709,2.4,6.0,8.4,1.4,0.5,1.5,1.3,2.3,10.9
12,2019,Al-Farouq Aminu,PF,28,POR,81,81,28.3,3.2,7.3,...,0.867,1.4,6.1,7.5,1.3,0.8,0.4,0.9,1.8,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2122,2016,Deron Williams,PG,31,DAL,65,63,32.4,4.9,11.9,...,0.869,0.3,2.6,2.9,5.8,0.9,0.2,2.3,2.4,14.1
2125,2016,Lou Williams,SG,29,LAL,67,35,28.5,4.2,10.3,...,0.830,0.5,2.1,2.5,2.5,0.9,0.3,1.6,1.6,15.3
2126,2016,Marvin Williams,PF,29,CHO,81,81,28.9,4.2,9.2,...,0.833,1.6,4.9,6.4,1.4,0.7,1.0,0.8,1.6,11.7
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,...,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1


<IPython.core.display.Javascript object>

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [22]:
# answer goes here
# feature columns: PTS: points, TRB: total rebounds, AST: assists, STL: steals, 
# BLK: blocks, TOV:turnovers, 3P%: 3 point percentage, 3P: 3 pointers

X = nba_df2[['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV', '3P']]
X


,PTS,TRB,AST,STL,BLK,TOV,3P
3,13.9,9.5,1.6,1.5,1.0,1.7,0.0
4,8.9,7.3,2.2,0.9,0.8,1.5,0.0
7,21.3,9.2,2.4,0.5,1.3,1.8,0.1
10,10.9,8.4,1.4,0.5,1.5,1.3,0.1
12,9.4,7.5,1.3,0.8,0.4,0.9,1.2
...,...,...,...,...,...,...,...
2122,14.1,2.9,5.8,0.9,0.2,2.3,1.5
2125,15.3,2.5,2.5,0.9,0.3,1.6,1.6
2126,11.7,6.4,1.4,0.7,1.0,0.8,1.9
2138,15.1,9.0,1.8,1.5,0.5,1.9,0.1


<IPython.core.display.Javascript object>

Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [23]:
# answer goes here
scale = StandardScaler()
X_scale = pd.DataFrame(scale.fit_transform(X), columns=X.columns)
X_scale


,PTS,TRB,AST,STL,BLK,TOV,3P
0,0.114638,1.612909,-0.617816,1.393220,0.824001,0.023511,-1.353529
1,-0.744985,0.768627,-0.330886,-0.050271,0.420162,-0.218831,-1.353529
2,1.386880,1.497780,-0.235242,-1.012598,1.429758,0.144681,-1.240535
3,-0.401136,1.190768,-0.713459,-1.012598,1.833596,-0.461172,-1.240535
4,-0.659023,0.845380,-0.761281,-0.290853,-0.387514,-0.945854,0.002389
...,...,...,...,...,...,...,...
799,0.149023,-0.919937,1.390695,-0.050271,-0.791352,0.750535,0.341369
800,0.355332,-1.073443,-0.187421,-0.050271,-0.589433,-0.097660,0.454362
801,-0.263596,0.423239,-0.713459,-0.531435,0.824001,-1.067025,0.793341
802,0.320947,1.421027,-0.522172,1.393220,-0.185595,0.265852,-1.240535


<IPython.core.display.Javascript object>

Run K-means clustering with K = 3 and print out the resulting centroids. When printing the centroids, transform the scaled centroids back into their corresponding unscaled values. What "archetypes" do you see?

In [24]:
# answer goes here
kmeans = KMeans(n_clusters=3)
kmeans.fit(X_scale)
centers = kmeans.cluster_centers_
centers_unscaled = pd.DataFrame(scale.inverse_transform(centers), columns=X.columns)
centers_unscaled.style.background_gradient()

# archetypes: cluster 1: Offensive (high points and steals), 
# cluster 0: defensive (high block and renounds)
# cluster 2: all-rounder 

,PTS,TRB,AST,STL,BLK,TOV,3P
0,13.207937,8.439683,1.867725,0.750794,1.203704,1.651323,0.441799
1,19.262944,5.253807,5.670558,1.342640,0.502030,2.701523,1.816244
2,10.402871,3.896651,2.045455,0.799043,0.357656,1.212679,1.248325


<IPython.core.display.Javascript object>

Experiment with different values of K. Do any further interesting archetypes come out?

In [25]:
# answer goes here
# k = 4
kmeans = KMeans(n_clusters=4)
kmeans.fit(X_scale)
centers = kmeans.cluster_centers_
centers_unscaled = pd.DataFrame(scale.inverse_transform(centers), columns=X.columns)
centers_unscaled.style.background_gradient()




,PTS,TRB,AST,STL,BLK,TOV,3P
0,19.918831,5.474026,6.194805,1.401299,0.509091,2.896104,1.785065
1,7.572845,4.417672,1.446121,0.650000,0.510345,0.980603,0.598707
2,13.535540,4.179094,2.662718,0.935889,0.342160,1.535192,1.714634
3,14.735878,9.096183,2.071756,0.803053,1.380916,1.809924,0.436641


<IPython.core.display.Javascript object>

In [26]:
# k = 5
kmeans = KMeans(n_clusters=5)
kmeans.fit(X_scale)
centers = kmeans.cluster_centers_
centers_unscaled = pd.DataFrame(scale.inverse_transform(centers), columns=X.columns)
centers_unscaled.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,3P
0,18.653731,10.537313,2.629851,0.974627,1.683582,2.310448,0.643284
1,13.606522,4.222826,2.651449,0.946377,0.350000,1.540580,1.724638
2,7.035366,3.430488,1.512805,0.618293,0.345122,0.892683,0.768902
3,19.512081,5.210738,6.248993,1.393289,0.451007,2.852349,1.788591
4,10.629730,7.083784,1.607432,0.708784,0.964189,1.350000,0.347297


<IPython.core.display.Javascript object>

In [ ]:
# The offensive and defensive archetypes are still clearly visible. 
#As someone who doesn’t watch basketball I can’t say if the 4th and 5th cluster
#represent new archetypes or if they are just subdivisions of the original 3rd archetype.